In [17]:
from pathlib import Path
import vtk
import pyvista as pv
import numpy as np

In [18]:
def read_vtu(filepath):
    stepCompression_6vtu = vtk.vtkXMLUnstructuredGridReader()
    stepCompression_6vtu.SetFileName(str(filepath))
    stepCompression_6vtu.Update()
    output_compression = stepCompression_6vtu.GetOutput()
    return output_compression

filepath = Path('C0003091_02/Step-Compression_4.vtu')
output_compression = read_vtu(filepath)
print(output_compression)

vtkUnstructuredGrid (0x7efe34059c40)
  Debug: Off
  Modified Time: 5197
  Reference Count: 1
  Registered Events: (none)
  Information: 0x55d64dd34d30
  Data Released: False
  Global Release Data: Off
  UpdateTime: 5238
  Field Data:
    Debug: Off
    Modified Time: 4716
    Reference Count: 1
    Registered Events: (none)
    Number Of Arrays: 0
    Number Of Components: 0
    Number Of Tuples: 0
  Number Of Points: 99828
  Number Of Cells: 95874
  Cell Data:
    Debug: Off
    Modified Time: 5192
    Reference Count: 1
    Registered Events: 
      Registered Observers:
        vtkObserver (0x55d64e04ab40)
          Event: 33
          EventName: ModifiedEvent
          Command: 0x55d64e1bf840
          Priority: 0
          Tag: 1
    Number Of Arrays: 37
    Array 0 name = COORD_Centroid
    Array 1 name = COORD_IntegrationPoints
    Array 2 name = LE_Centroid
    Array 3 name = LE_IntegrationPoints
    Array 4 name = S_Centroid
    Array 5 name = S_IntegrationPoints
    Array 6 n

In [39]:
def decomposition(JacobianArray):
    Terms = JacobianArray.shape[-1]
    ArrayShape = JacobianArray.shape[:-1]

    if Terms == 4:

        SphericalCompression = np.zeros(ArrayShape)
        IsovolumicDeformation = np.zeros(ArrayShape)
        # HydrostaticStrain = np.zeros(ArrayShape)
        # VonMises_Strain = np.zeros(ArrayShape)
        # MaxShear = np.zeros(ArrayShape)

        # ProcessLength = ArrayShape[0] * ArrayShape[1]
        # Progress = 0
        for j in range(0, ArrayShape[0]):
            for i in range(0, ArrayShape[1]):
                F_d = JacobianArray[j, i, :].reshape((2,2))

                ## Unimodular decomposition of F
                J = np.linalg.det(F_d)
                SphericalCompression[j, i] = J

                if J > 0:
                    F_tilde = J ** (-1 / 3) * F_d
                    Norm_F_tilde = np.linalg.norm(F_tilde)
                else:
                    Norm_F_tilde = 0.0

                IsovolumicDeformation[j, i] = Norm_F_tilde

                # ## Optional: decomposition of F_tilde
                # R_tilde, U_tilde = polar(F_tilde)
                # Norm_U_tilde = np.sqrt(np.sum(U_tilde ** 2))

                ## Hydrostatic and deviatoric strain
                # I_d = np.matrix(np.eye(F_d.shape[0]))
                # E = 1/2 * (F_d.T * F_d - I_d)
                # Hydrostatic_E = -1/3 * np.trace(E) * I_d
                # Deviatoric_E = E - Hydrostatic_E
                #
                # HydrostaticStrain[k,j,i] = Hydrostatic_E[0,0]
                # MaxShear[k,j,i] = E.diagonal().max() - E.diagonal().min()
                #
                # VM_Strain = np.sqrt(3/2) * np.linalg.norm(Deviatoric_E)
                # VonMises_Strain[k,j,i] = VM_Strain

                # Progress += 1
                # ProgressNext(Progress/ProcessLength*20)

    elif Terms == 9:

        SphericalCompression = np.zeros(ArrayShape)
        IsovolumicDeformation = np.zeros(ArrayShape)
        # HydrostaticStrain = np.zeros(ArrayShape)
        # VonMises_Strain = np.zeros(ArrayShape)
        # MaxShear = np.zeros(ArrayShape)

        # ProcessLength = ArrayShape[0] * ArrayShape[1] * ArrayShape[2]
        # Progress = 0
        for k in range(0, ArrayShape[0]):
            for j in range(0, ArrayShape[1]):
                for i in range(0, ArrayShape[2]):

                    F_d = JacobianArray[k, j, i, :].reshape((3, 3))

                    ## Unimodular decomposition of F
                    J = np.linalg.det(F_d)
                    SphericalCompression[k, j, i] = J

                    if J > 0:
                        F_tilde = J ** (-1 / 3) * F_d
                        Norm_F_tilde = np.linalg.norm(F_tilde)
                    else:
                        Norm_F_tilde = 0.0

                    IsovolumicDeformation[k, j, i] = Norm_F_tilde

                    # ## Optional: decomposition of F_tilde
                    # R_tilde, U_tilde = polar(F_tilde)
                    # Norm_U_tilde = np.sqrt(np.sum(U_tilde ** 2))

                    ## Hydrostatic and deviatoric strain
                    # I_d = np.matrix(np.eye(F_d.shape[0]))
                    # E = 1/2 * (F_d.T * F_d - I_d)
                    # Hydrostatic_E = -1/3 * np.trace(E) * I_d
                    # Deviatoric_E = E - Hydrostatic_E
                    #
                    # HydrostaticStrain[k,j,i] = Hydrostatic_E[0,0]
                    # MaxShear[k,j,i] = E.diagonal().max() - E.diagonal().min()
                    #
                    # VM_Strain = np.sqrt(3/2) * np.linalg.norm(Deviatoric_E)
                    # VonMises_Strain[k,j,i] = VM_Strain
                    
                    # Progress += 1
                    # ProgressNext(Progress/ProcessLength*20)

    return SphericalCompression, IsovolumicDeformation

arrays = output_compression.GetCellData()
centroid_arrays = {arrays.GetArrayName(i): arrays.GetArray(i) for i in range(arrays.GetNumberOfArrays()) if 'Centroid' in arrays.GetArrayName(i)}
F11 = centroid_arrays['SDV_F11_Centroid']
F12 = centroid_arrays['SDV_F12_Centroid']
F13 = centroid_arrays['SDV_F13_Centroid']
F21 = centroid_arrays['SDV_F21_Centroid']
F22 = centroid_arrays['SDV_F22_Centroid']
F23 = centroid_arrays['SDV_F23_Centroid']
F31 = centroid_arrays['SDV_F31_Centroid']
F32 = centroid_arrays['SDV_F32_Centroid']
F33 = centroid_arrays['SDV_F33_Centroid']
F = np.array([F11, F12, F13, F21, F22, F23, F31, F32, F33]).reshape(3, 3, -1)

'''
# Build, decompose and write deformation gradient
F = np.zeros((len(Z),len(Y),len(X),9))
F[:,:,:,0] = 1.0
F[:,:,:,4] = 1.0
F[:,:,:,8] = 1.0
SC, ID = [], []
for Index in ElData.index:
    Element = ElData.loc[Index]
    X_Index = list(X).index(Element['X'])
    Y_Index = list(Y).index(Element['Y'])
    Z_Index = list(Z).index(Element['Z'])
    
    F[Z_Index,Y_Index,X_Index] = Element[Fs].values

SphericalCompression, IsovolumicDeformation = decomposition(F)

for Index in ElData.index:
    Element = ElData.loc[Index]
    X_Index = list(X).index(Element['X'])
    Y_Index = list(Y).index(Element['Y'])
    Z_Index = list(Z).index(Element['Z'])
    SC.append(SphericalCompression[Z_Index,Y_Index,X_Index])
    ID.append(IsovolumicDeformation[Z_Index,Y_Index,X_Index])
SC, ID = decomposition(JacobianArray)
'''

(3, 3, 95874)


"\n# Build, decompose and write deformation gradient\nF = np.zeros((len(Z),len(Y),len(X),9))\nF[:,:,:,0] = 1.0\nF[:,:,:,4] = 1.0\nF[:,:,:,8] = 1.0\nSC, ID = [], []\nfor Index in ElData.index:\n    Element = ElData.loc[Index]\n    X_Index = list(X).index(Element['X'])\n    Y_Index = list(Y).index(Element['Y'])\n    Z_Index = list(Z).index(Element['Z'])\n    \n    F[Z_Index,Y_Index,X_Index] = Element[Fs].values\n\nSphericalCompression, IsovolumicDeformation = decomposition(F)\n\nfor Index in ElData.index:\n    Element = ElData.loc[Index]\n    X_Index = list(X).index(Element['X'])\n    Y_Index = list(Y).index(Element['Y'])\n    Z_Index = list(Z).index(Element['Z'])\n    SC.append(SphericalCompression[Z_Index,Y_Index,X_Index])\n    ID.append(IsovolumicDeformation[Z_Index,Y_Index,X_Index])\nSC, ID = decomposition(JacobianArray)\n"